# 1. Overview

Based on the descriptive and exploratory analysis done in notebook 00_data_understanding, this Python Script will work on preprocessing the data, preparing it so that we can then work on the model training in the future.

# 2. Data Understanding

## 2.1 Data Description

This file will use the df_train_transform excel sheet created in the previous notebook: 00_data_understanding

## 2.2 Import Necessary Libraries

In [1]:
# pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder
from category_encoders import TargetEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split



## 2.3 Functions

# 3. Code

## 3.1 Import the database

In [3]:
df = pd.read_excel('df_train_transform.xlsx')
df.head()

,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped,status_group
0,6000.0,1390,109,lake nyasa,iringa,1.0,0.0,gravity,usergroup,annually,good,enough,spring,communal standpipe,government,functional
1,0.0,1399,280,lake victoria,mara,NaN,1.0,gravity,usergroup,never pay,good,insufficient,rainwater harvesting,communal standpipe,other,functional
2,25.0,686,250,pangani,manyara,1.0,1.0,gravity,usergroup,per bucket,good,enough,dam,communal standpipe multiple,government,functional
3,0.0,263,58,ruvuma southern coast,mtwara,1.0,1.0,submersible,usergroup,never pay,good,dry,borehole,communal standpipe multiple,government,non functional
4,0.0,0,0,lake victoria,kagera,1.0,1.0,gravity,other,never pay,good,seasonal,rainwater harvesting,communal standpipe,other,functional


## 3.2 Dealing with null values

In [4]:
# For train data
(df.isna().sum()/len(df))*100

amount_tsh                   0.000000
gps_height                   0.000000
population                   0.000000
basin                        0.000000
region                       0.000000
public_meeting               5.612795
permit                       5.144781
extraction_type_class        0.000000
management_group             0.000000
payment_type                 0.000000
quality_group                0.000000
quantity_group               0.000000
source_type                  0.000000
waterpoint_type              0.000000
scheme_management_grouped    0.000000
status_group                 0.000000
dtype: float64

### Column 'public_meeting'

In [5]:
df["public_meeting"].value_counts(normalize=True)

1.0    0.909838
0.0    0.090162
Name: public_meeting, dtype: float64

In [6]:
# Given that the null values are only 6%, lets replace them with the mode

# Calculate the mode of the 'public_meeting' column
permit_mode = df['public_meeting'].mode()[0]

# Fill missing values in 'public_meeting' with the mode
df['public_meeting'].fillna(permit_mode, inplace=True)

# Verify if all NA values are filled
print(df['public_meeting'].value_counts(normalize=True))

1.0    0.914899
0.0    0.085101
Name: public_meeting, dtype: float64


### Column 'permit'

In [7]:
df["permit"].value_counts(normalize=True)

1.0    0.68955
0.0    0.31045
Name: permit, dtype: float64

In [8]:
# Given that the null values are only 5%, lets replace them with the mode

# Calculate the mode of the 'permit' column
permit_mode = df['permit'].mode()[0]

# Fill missing values in 'permit' with the mode
df['permit'].fillna(permit_mode, inplace=True)

# Verify if all NA values are filled
print(df['permit'].value_counts(normalize=True))

1.0    0.705522
0.0    0.294478
Name: permit, dtype: float64


## 3.3 Class Imbalance checking

In [9]:
# Check class distribution in y_train
print("Class distribution of status_group:")
print(df['status_group'].value_counts(normalize=True))

Class distribution of status_group:
functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64


We decide to group together into a same class functional needs repair and functional. In this way, we have a binary classification problem

In [10]:
# Replace 'functional needs repair' with 'functional'
df['status_group'] = df['status_group'].replace('functional needs repair', 'functional')

# Verify changes by checking the class distribution again in y_train and y_test
print("Class distribution in y_train after replacement:")
print(df['status_group'].value_counts(normalize=True))

Class distribution in y_train after replacement:
functional        0.615758
non functional    0.384242
Name: status_group, dtype: float64


## 3.4 Define predictor and target variables

In [11]:
y = df['status_group']
X = df.drop('status_group', axis=1)

## 3.5 Do a train test split

In [12]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## 3.6 Doing target enconder on the categorical columns

Let's perform a one hot enconder on the categorical columns that have less than 6 categories

In [13]:
# Identifying categorical columns
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns

# Printing the list of categorical columns
print("Categorical columns in X_train:")
print(categorical_columns)

Categorical columns in X_train:
Index(['basin', 'region', 'extraction_type_class', 'management_group',
       'payment_type', 'quality_group', 'quantity_group', 'source_type',
       'waterpoint_type', 'scheme_management_grouped'],
      dtype='object')


In [14]:
X_train['management_group'].value_counts()

usergroup     42018
commercial     2869
parastatal     1413
other           764
unknown         456
Name: management_group, dtype: int64

In [15]:
X_train['payment_type'].value_counts()

never pay     20318
per bucket     7223
monthly        6574
unknown        6521
on failure     3154
annually       2886
other           844
Name: payment_type, dtype: int64

Given that most of the categorical columns have more than 6 values, we are going to perfom a target enconder on these categorical columns

**X_train**

In [16]:
# Convert 'status_group' to a categorical type with a numeric representation

# First, check if 'y_train' needs to be converted
if y_train.dtype == 'object':
    y_train = y_train.astype('category').cat.codes

# Check if 'y_test' needs to be converted
if y_test.dtype == 'object':
    y_test = y_test.astype('category').cat.codes

# Capture categorical columns from X_train for encoding
categorical_columns = X_train.select_dtypes(include=['object', 'category']).columns

# Initialize the TargetEncoder
encoder = TargetEncoder(cols=categorical_columns)

# Fit and transform the categorical columns in X_train
X_train[categorical_columns] = encoder.fit_transform(X_train[categorical_columns], y_train)

# Transform the categorical columns in X_test using the same encoder
X_test[categorical_columns] = encoder.transform(X_test[categorical_columns])

# Display the DataFrame to check the results
X_train.head()

C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped
3607,50.0,2092,160,0.346722,0.315956,1.0,1.0,0.300187,0.387715,0.277862,0.357050,0.379627,0.301175,0.298881,0.218391
50870,0.0,0,0,0.346722,0.443875,1.0,1.0,0.309484,0.387715,0.475440,0.357050,0.274700,0.447489,0.324167,0.410441
20413,0.0,0,0,0.485901,0.398196,1.0,0.0,0.805243,0.387715,0.475440,0.357050,0.274700,0.447489,0.821499,0.410441
52806,0.0,0,0,0.311216,0.398196,1.0,1.0,0.300187,0.387715,0.226650,0.357050,0.379627,0.343784,0.298881,0.410441
50091,300.0,1023,120,0.432348,0.398697,1.0,1.0,0.805243,0.387715,0.308180,0.483585,0.274700,0.447489,0.821499,0.410441


**X_test**

In [18]:
# Transform the categorical columns in X_test using the same encoder used for X_train
X_test[categorical_columns] = encoder.transform(X_test[categorical_columns])

# Display the DataFrame to check the results for X_test
X_test.head()

C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped
2980,0.0,0,0,0.384091,0.384091,1.0,1.0,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091
5246,0.0,0,0,0.384091,0.384091,1.0,1.0,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091
22659,10.0,1675,148,0.384091,0.384091,1.0,1.0,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091
39888,0.0,0,0,0.384091,0.384091,0.0,0.0,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091
13361,50.0,1109,235,0.384091,0.384091,1.0,1.0,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091,0.384091


## 3.7 Dealing with numerical columns

**X_train**

In [19]:
# Capture numerical columns
numerical_columns = X_train.select_dtypes(include=['int64', 'float64']).columns

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the numerical columns
scaler.fit(X_train[numerical_columns])

X_train[numerical_columns] = scaler.transform(X_train[numerical_columns])

# Display the DataFrame to check the results
X_train.head()

<ipython-input-19-654df8225f2e>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[numerical_columns] = scaler.transform(X_train[numerical_columns])
C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped
3607,-0.084999,2.053863,-0.041306,-0.540016,-0.633090,0.306797,0.648002,-0.521411,0.159294,-0.897587,-0.295474,-0.021169,-1.112570,-0.510890,-2.699527
50870,-0.100621,-0.965049,-0.379739,-0.540016,0.555492,0.306797,0.648002,-0.463637,0.159294,0.771866,-0.295474,-0.518708,0.850673,-0.359301,0.429287
20413,-0.100621,-0.965049,-0.379739,1.471270,0.131062,0.306797,-1.543206,2.617222,0.159294,0.771866,-0.295474,-0.518708,0.850673,2.622191,0.429287
52806,-0.100621,-0.965049,-0.379739,-1.053126,0.131062,0.306797,0.648002,-0.521411,0.159294,-1.330306,-0.295474,-0.021169,-0.540840,-0.510890,0.429287
50091,-0.006889,0.511216,-0.125914,0.697368,0.135714,0.306797,0.648002,2.617222,0.159294,-0.641415,1.087153,-0.518708,0.850673,2.622191,0.429287


**X_test**

In [20]:
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Display the DataFrame to check the results
X_test.head()

<ipython-input-20-f81a08083e1e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])
C:\Users\Usuario\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped
2980,-0.100621,-0.965049,-0.379739,0.0,0.0,0.306797,0.648002,0.000002,7.320293e-15,0.0,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.0
5246,-0.100621,-0.965049,-0.379739,0.0,0.0,0.306797,0.648002,0.000002,7.320293e-15,0.0,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.0
22659,-0.097497,1.452101,-0.066689,0.0,0.0,0.306797,0.648002,0.000002,7.320293e-15,0.0,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.0
39888,-0.100621,-0.965049,-0.379739,0.0,0.0,-3.259489,-1.543206,0.000002,7.320293e-15,0.0,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.0
13361,-0.084999,0.635320,0.117334,0.0,0.0,0.306797,0.648002,0.000002,7.320293e-15,0.0,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.0


## 3.8 Concatenate train on one side and test on the other

In [21]:
# Concatenate all train
df_train = pd.concat([X_train, y_train], axis=1)

# Concatenate all test
df_test = pd.concat([X_test, y_test], axis=1)

# Create a label column
df_train['is_test'] = 0
df_test['is_test'] = 1

## 3.9 Concatenate everything in one dataframe

In [22]:
data_processed = pd.concat([df_train,df_test], axis=0)

# Reset index
data_processed = data_processed.reset_index(drop=True)

# Rename column 0 to status_group
data_processed = data_processed.rename(columns={0: 'status_group'})

data_processed

,amount_tsh,gps_height,population,basin,region,public_meeting,permit,extraction_type_class,management_group,payment_type,quality_group,quantity_group,source_type,waterpoint_type,scheme_management_grouped,status_group,is_test
0,-0.084999,2.053863,-0.041306,-0.540016,-0.633090,0.306797,0.648002,-0.521411,1.592945e-01,-0.897587,-2.954744e-01,-2.116901e-02,-1.112570e+00,-0.510890,-2.699527,0,0
1,-0.100621,-0.965049,-0.379739,-0.540016,0.555492,0.306797,0.648002,-0.463637,1.592945e-01,0.771866,-2.954744e-01,-5.187079e-01,8.506732e-01,-0.359301,0.429287,0,0
2,-0.100621,-0.965049,-0.379739,1.471270,0.131062,0.306797,-1.543206,2.617222,1.592945e-01,0.771866,-2.954744e-01,-5.187079e-01,8.506732e-01,2.622191,0.429287,1,0
3,-0.100621,-0.965049,-0.379739,-1.053126,0.131062,0.306797,0.648002,-0.521411,1.592945e-01,-1.330306,-2.954744e-01,-2.116901e-02,-5.408401e-01,-0.510890,0.429287,1,0
4,-0.006889,0.511216,-0.125914,0.697368,0.135714,0.306797,0.648002,2.617222,1.592945e-01,-0.641415,1.087153e+00,-5.187079e-01,8.506732e-01,2.622191,0.429287,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,-0.038133,1.596408,0.741319,0.000000,0.000000,0.306797,-1.543206,0.000002,7.320293e-15,0.000000,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.000000,0,1
59396,0.055600,1.704639,-0.062458,0.000000,0.000000,0.306797,0.648002,0.000002,7.320293e-15,0.000000,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.000000,0,1
59397,-0.100621,-0.965049,-0.379739,0.000000,0.000000,0.306797,-1.543206,0.000002,7.320293e-15,0.000000,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.000000,1,1
59398,-0.100621,-0.038596,-0.377623,0.000000,0.000000,0.306797,0.648002,0.000002,7.320293e-15,0.000000,-1.022441e-09,2.632229e-16,1.021931e-12,-0.000058,0.000000,0,1


# 4. Export the data

In [23]:
data_processed.to_excel('df_data_processed.xlsx', index=False)